# Attrs

## Intro

* Attrs is a better way to create classes by [Hynek Schlawack](https://hynek.me/)
* [Attrs Pip](https://pypi.org/project/attrs/)
* [Attrs Docs](https://www.attrs.org/en/stable/)
* [Attrs Extensions](https://github.com/python-attrs/attrs/wiki/Extensions-to-attrs)
* [Relevant Video Lecture](https://www.youtube.com/watch?v=COMRNKAVesI&t=945s)

* When you create classes you normally use [special methods](https://docs.python.org/3/reference/datamodel.html#specialnames) (Dunder Methods).
    * Implementing things like `__init__` and `__add__` tend to generate boilerplate.
    * So instead of using the standard method names you can use `attrs` as a shortcut.

### README Example

In [1]:
import attr

# Create a class (SomeClass) using (attr.s) decorator
# Use (attrs.ib) function to create the attributes of the class.
# Define the default values for (a_number = 42) and (list_of_numbers =[]).
@attr.s
class SomeClass(object):
    a_number = attr.ib(default=42)
    list_of_numbers = attr.ib(factory=list)
    def hard_math(self, another_number):
        return self.a_number + sum(self.list_of_numbers) * another_number

# Instead of (attr.s) you can use (attrs).
# Instead of (attr.ib) you can use (attrib).
    
# Now you can instance the class and use its method
sc = SomeClass(1, [1, 2, 3])
r = sc.hard_math(3)
print("Hard Math Result: " + str(r))

# Also attrs creates comparison methods by default
compare1 = (sc == SomeClass(1, [1, 2, 3]))
print("Is the object sc equal: " + str(compare1))
compare2 = (sc != SomeClass(2, [3, 2, 1]))
print("Is the object sc different: " + str(compare2))

# We can get the properties of an object as a dict
dictSC = attr.asdict(sc)
print("Properties of sc as dict: " + str(dictSC))

# Attrs can even create shorthand classes with (make_class) function.
ShortClass = attr.make_class("C", ["a", "b"])
short = ShortClass("foo", "bar")
print("Object (short) initial values: " + str(short))

Hard Math Result: 19
Is the object sc equal: True
Is the object sc different: True
Properties of sc as dict: {'a_number': 1, 'list_of_numbers': [1, 2, 3]}
Object (short) initial values: C(a='foo', b='bar')


## Alternatives

* You could use **Tuples** to define the attributes of a class.
    * However this becomes really hard to debbug.
    * Additional attributes need to fix unpacking declarations.
    * You need to index attributes in order to access them.
    
```python
# Using Attrs
Point(x=1, y=2)
# Using Tupples
(1, 2)

# Using Attrs
Customer(id=42, reseller=23, first_name="Jane", last_name="John")
# Using Tupples
(42, 23, "Jane", "John")
```

* You could also use **Named Tuples** from `collections.namedtuples`.
    * They are still a Tuple hack that just imitates a class.
    * They are not type sensitive when comparing.
    * Subclass of Tuple so **always** indexable and interable.
    * Always adds Tuple methods to every instance.
    
```python
# Using Attrs
C1 = attr.make_class("C1", ["a"])
C2 = attr.make_class("C2", ["a"])
i1 = C1(1) 
i2 = C2(1)
# Same attribute value
compare1 = (i1.a == i2.a)
print(compare1)
>> True
# But different Class
compare2 = (i1 == i2)
print(compare2)
>> False

# Using Namedtuples
from collections import namedtuple
NT1 = namedtuple("NT1", "a")
NT2 = namedtuple("NT2", "b")
t1 = NT1(1)
t2 = NT2(1)
# Same tuple type, therefore same "class"
compare1 = (t1 == t2 == (1,))
print(compare1)
>> True
```

* A recent option is the use of **Data Classes** from PEP 557.
    * They are the *official* version of attrs added on 3.7.
    * But they are missing advanced validators, converters, slots etc.
    * Also Attrs has a quicker development cycle.
    * Posible fallback if pip install is not an option.
   
   
* In the end Attrs is the best option around.
    * You can even manually override the attributes if you really need to.

```python
# Override the default value for the attr (__repr__) special method. 
@attr.s(repr=False)
class SmartClass(object):
    a = attr.ib()
    b = attr.ib()
    def __repr__(self):
        return "<SmartClass(a=%d)>" % (self.a,)
    
print(SmartClass(1, 2))
>> <SmartClass(a=1)>
```

## Use Examples

### Basics

* Simplest attr empty class

```python
@attr.s
    class Empty(object):
        pass

# The class has a __repr__ string.
print(Empty())
# Two instances are of the same class.
print(Empty() == Empty())
# However the two are independent.
print(Empty() is Empty())

>> Empty()
>> True
>> False
```

* Attrs class with attributes

```python
@attr.s
class Coordinates(object):
    x = attr.ib()
    y = attr.ib()

# You can use __init__ using positional arguments.
c1 = Coordinates(1, 2)

# Or if you prefer you can use keyword arguments.
c1 = Coordinates(x = 2, y = 1)

print(c1)
print(c2)
print(c1 == c2)

>> Coordinates(x=1, y=2)
>> Coordinates(x=2, y=1)
>> False
```

* Instead of attr.s and attr.ib you can use attrs and attrib.

```python
from attr import attrs, attrib
@attrs
class SeriousCoordinates(object):
    x = attrib()
    y = attrib()
print(SeriousCoordinates(1, 2))
>> SeriousCoordinates(x=1, y=2)

# Are the fields of both classes the same?
print(attr.fields(Coordinates) == attr.fields(SeriousCoordinates))
>> True
```

* Attrs intentionally ignores the private attribute underscore.
    * This is because *there is no such thing as private attributes*.
    * At the code level that underscore is just syntactic sugar.
    * However, if you keep on using it **remember** that it is ignored!

```python
import inspect, attr
@attr.s
class C(object):
    _x = attr.ib()
    
# Look, it doesn't need the underscore!
i = C( x = 1 )

print(i)
>> C(_x=1)

# The signature does not contain the underscore
print(inspect.signature(C.__init__))
>> <Signature (self, x) -> None>

# ERROR because the valid name (_1)
# transformed into the invalid name (1)
@attr.s
class C(object):
    _1 = attr.ib()
>> SyntaxError: invalid syntax
```


* There is another way to define attributes that is used to enhance other people classes.
 
```python
# Another person defines a class with an (x) attribute
class SomethingFromSomeoneElse(object):
    def __init__(self, x):
        self.x = x
        
# You can fill the rest with attr using 
SomethingFromSomeoneElse = attr.s(
    these={
        "x": attr.ib()
    }, init=False)(SomethingFromSomeoneElse)

# Now it is as if it was created with Attrs.
print(SomethingFromSomeoneElse(1))
>> SomethingFromSomeoneElse(x=1)
```

* Subclassing is bad practice, but it still works with attrs.
    * The order of arguments with subclassing is defined by the [Method Resolution Order](https://www.python.org/download/releases/2.3/mro/).
    * Instead of subclassing and inheritance use composition.
        * Use classes as types with their own methods to isolate their tasks.
        * Explicitly call for the methods and attributes of other classes.
        * Types become layers of your architecture increasing fault intolerance.
        * Use interfaces, abstract, and factories to break classes into pieces.
        
```python
# Example of subclassing with attr.
# Class A has attribute (a) and can return its value with (get_a)
@attr.s
class A(object):
    a = attr.ib()
    def get_a(self):
        return self.a
    
# Class B has attribute (b)
@attr.s
class B(object):
    b = attr.ib()

# Class C inherites ALL from (A) and (B) and has attribute (c)
@attr.s
class C(A, B):
    c = attr.ib()

i = C(1, 2, 3)

# Therefore C has its a, b, c and (get_a)
print(i)
print(i == C(1, 2, 3))
print(i.get_a())
>> C(a=1, b=2, c=3)
>> True
>> 1
```

* In python 3 classes inside other classes are reflected in (__repr__).
    * The `repr_ns` option defines the repr of the upper level.
    * This is useful because it also works on Python 2.
    
```python
# Define a class
@attr.s
class C(object):
    # Define the (repr) of the upper level class
    @attr.s(repr_ns = "G")
    class D(object):
        pass
# When you call the inner class it will use repr_ns instead of repr
i = C.D()
print(i)
>> G.D()
```

* Python initialization happens at `init`.
* Initialization should have as little logic as possible.
* Passing complex objects couples classes and creates difficult testing.
* So if you use an ORM (Object Relational Mapping):
    * Where you get your attribute data from a database.
    * Attr recommends that you fallow this way of initialization.
    * This way you minimize fake classes and create smart helpers.
       
```python
# (Row is a Database Row)
# Bad ORM pattern
class Point(object):
    def __init__(self, row):
        self.x = row.x
        self.y = row.y
newPoint = Point(dRow)

# Another bad ORM pattern
newPoint2 = Point(**row.attributes)

# Good ORM pattern
@attr.s
class Point(object):
    x = attr.ib()
    y = attr.ib()
    @classmethod
    def from_row(cls, row):
        return cls(row.x, row.y)
newPoint = Point.from_row(dRow)
```

### Keyword Only

* On python 3 you can restrict you attrs arguments to be *keyword-only*.
    * Note that if you add `init=False` then this restriction is ignored.

```python
# Restrict one argument to be keyword only
@attr.s
class A:
    a = attr.ib(kw_only=True)

# Printing without keyword gets error
print(A())
>> TypeError: __init__() missing 1 required keyword-only argument: 'a'
print(A(1))
>> TypeError: __init__() takes 1 positional argument but 2 were given
    
# Printing with correct keyword argument
print(A(a=1))
>> A(a=1)

# Restrict all arguments to be keyword only

@attr.s(kw_only=True)
class B:
    b = attr.ib()
    c = attr.ib()
    
# Printing without keywords gets error
print(B(1,2))
>> TypeError: __init__() takes 1 positional argument but 3 were given
    
# Printing with correct keyword argument
print(B(b=1, c=2))
>> B(b=1, c=2)
```

* If you define a class with attributes that have default values, and then a subclass:
    * You get an error if you try to add attributes without default values to the subclass.
 
```python
# Base class with default value attributes
@attr.s
class A:
    a = attr.ib(default=0)
# Subclass without default value attributes
@attr.s
class B(A):
    b = attr.ib()

>> ValueError: No mandatory attributes allowed after an attribute with a default 
```

* Keyword Only is a way to successfully add attributes to a subclass without default values.
    
```python
# Base class with default value attributes
@attr.s
class A:
    a = attr.ib(default=0)
# Subclass without default value attributes
@attr.s
class B(A):
    b = attr.ib(kw_only=True)

print(B(b=1))
>> B(a=0, b=1)
```

### Converting

* When you have a class it can be convenient to transform it into a dict.
    * For example, to serialize it into JSON using the `asdict` method.
    
```python
@attr.s
class Coordinates(object):
    x = attr.ib()
    y = attr.ib()

print(attr.asdict(Coordinates(x=1, y=2)))
>> {'x': 1, 'y': 2}
```

* To ignore one attribute in the transformation we use a callback.

```python
@attr.s
class UserList(object):
    users = attr.ib()
    
@attr.s
class User(object):
    email = attr.ib()
    password = attr.ib()

# Create a list of users, each with an email and password.
i = UserList([User("jane@doe.invalid", "s33kred"), User("joe@doe.invalid", "p4ssw0rd")])
# Filter the attribute with the name (identifier) "password"
d = attr.asdict( i, filter=lambda attr, value: attr.name != "password")

print(d)
>> {'users': [{'email': 'jane@doe.invalid'}, {'email': 'joe@doe.invalid'}]}
```

* Use the method `exclude` or `include` to filter multiple attributes.

```python
@attr.s
class User(object):
    login = attr.ib()
    password = attr.ib()
    idUser = attr.ib()

# Create a user with login, password and id
j = User("jane", "s33kred", 42)
# Exclude the attribute with name "User"
fieldToExclude = attr.fields(User).password
# And also exclude the (int) attribute "idUser"
d = attr.asdict( j, filter=attr.filters.exclude(fieldToExclude, int))
# That leaves just the login attribute "jane"
print(d) 
>> {'login': 'jane'}

@attr.s
class C(object):
    x = attr.ib()
    y = attr.ib()
    z = attr.ib()

# Create a C instance with x, y, z
ci = C("foo", "2", 3)
# Only include the attribute with name "x" and the (int) attribute "z"
d = attr.asdict(ci, filter=attr.filters.include(attr.fields(C).x, int))
print(d)
>> {'x': 'foo', 'z': 3}
```

* Instead of a dict we may want to create a tuple with `astuple`.

```python
import sqlite3
@attr.s
class Foo:
    a = attr.ib()
    b = attr.ib()
    
fooI = Foo(2, 3)

# Create a table (foo) with columns (x) as ID and (y) as values.
# INSERT values (a) and (b) from instance fooI with a Tuple.
# Create another instance (foo2I) with the values stored on (foo).
with sqlite3.connect(":memory:") as conn:
   c = conn.cursor()
   c.execute("CREATE TABLE foo (x INTEGER PRIMARY KEY ASC, y)") 
   c.execute("INSERT INTO foo VALUES (?, ?)", attr.astuple(fooI)) 
   foo2I = Foo(*c.execute("SELECT x, y FROM foo").fetchone())
    
>> <sqlite3.Cursor object at ...>

# Comparing the content of fooI and foo2I they are the same.
print( foo == foo2 )
>> True
```

### Validators

* Ideally class initializers should only initialize your instances.
    * However Attr can apply validators for its class attributes.
    * You can define validators with a decorator or a callable.
    * Decorator validator works ONLY if the attribute has an attr.ib assigned.
    * Like `attr.validators.instance_of(int)` attr includes common validators
    
```python
# Using a decorator
@attr.s
class C(object):
    x = attr.ib()
    @x.validator
    def check(self, attribute, value):
        if value > 42:
            raise ValueError("X must be smaller or equal to 42")

# You raise a ValueError if the attribute value is > 42
print(C(43))
>> ValueError: "X must be smaller or equal to 42"
  
    
# Using a callable
def x_smaller_than_y(instance, attribute, value):
    if value >= instance.y:
        raise ValueError("'x' has to be smaller than 'y'")
     
@attr.s
class C(object):
    x = attr.ib(validator=[attr.validators.instance_of(int), x_smaller_than_y])
    y = attr.ib()

# You raise a ValueError if X is >= Y
# Also raise a ValueError if X is nor an Int
print(C(x=3, y=3))
>> ValueError: "'x' has to be smaller than 'y'"
  
    
# Using Both
@attr.s
class C(object):
    x = attr.ib(validator=attr.validators.instance_of(int))
    @x.validator
    def fits_byte(self, attribute, value):
        if not 0 <= value < 256:
            raise ValueError("Value out of bounds (byte 0-255)")

# You raise a TypeError if X is not an INT
print(C("128"))
>> TypeError: "'x' must be <class 'int'> (got '128' that is a <class 'str'>)."

# You raise a ValueError if X is out of bounds for a byte.
print(C(300))
>> ValueError: "Value out of bounds (byte 0-255)"
```

* If you make changes to those attributes you can call `attr.validate` at any time.
    * This will make the validation of the attribute again 
    
```python
i = C(200)
i.x = 300
attr.validate(i)
>> ValueError: "'x' has to be smaller than 'y'!"
```

* Finally you can disable or enable validators globally with `set_run_validators()`.

```python
attr.set_run_validators(False)
print(C("128"))
C(x='128')
attr.set_run_validators(True)
print(C("128"))
>> TypeError: "'x' must be <class 'int'> (got '128' that is a <class 'str'>)."
```

### Convertions, Metadata, Slots

* Attributes can have a converter function defined.
    * This will be called with the value passed to get a new value.
    * They normalize the values coming in before validators.
    
```python
@attr.s
class C(object):
    x = attr.ib(converter=int)

i = C("1")

print(type(i.x))
>> <class 'int'>
```

* All attributes may include arbitrary metadata.

```python
@attr.s
class C(object):
    x = attr.ib(metadata={'my_metadata': 1})

i = C("Hi")

# The metadata is really a mappingproxy
print(i.x.metadata)
>> mappingproxy({'my_metadata': 1})

# But works the same as a dictionary.
print(i.x.metadata['my_metadata'])
>> 1
```

* Slotted classes are better for some thin example on CPython. 
    * Defining `slots` is tedious, in attr you only need to set `slots=True`.
    
```python
# Create a slotted class.
@attr.s(slots=True)
class Coordinates(object):
    x = attr.ib()
    y = attr.ib()
```
    
* You can add immutability to your instances with `frozen=True`.
    * Especially popular in functional programming.
    * Note that python doesn't have TRUE immutability.
    
```python
# Create an immutable or frozen class.
@attr.s(frozen=True)
class C(object):
    x = attr.ib()
i = C(1)
i.x = 2
>> attr.exceptions.FrozenInstanceError: "can't set attribute"
```

* Use `attr.evolve` to create new instances based on frozen instances.

```python
@attr.s(frozen=True)
class C(object):
    x = attr.ib()
    y = attr.ib()
i1 = C(1, 2)
i2 = attr.evolve(i1, y=3)
print(i1)
print(i2)
>> C(x=1, y=2)
>> C(x=1, y=3)
```

### Other Uses

* To create classes programmatically use `attr.make_class`.

```python
@attr.s
class C1(object):
    x = attr.ib()
    
C2 = attr.make_class("C2", ["x"])

print(attr.fields(C1) == attr.fields(C2))
>>> True
```

* For more flexibility you can pass a dictionary to `make_class`.

```python
C3 = attr.make_class("C2", {"x": attr.ib(default=42)}, repr=False)

i = C3()

# Repr = False so no auto-representation from attr.
print(i)
print(i.x)
>> <__main__.C3 object at>
>> 42
```

* To dynamically make a subclass add the `bases` argument.

```python
class D(object):
    def __eq__(self, other):
        return True

E = attr.make_class("E", {}, bases=(D,), cmp=False)

# Since E is a subclass of D then E instances are also of D.
print(isinstance(E(), D))
>> True
```

* You can use `repr` with attr.ib to change some attributes.
    * For no repr use False, for custom repr use a custom callable.

```python
@attr.s
class C(object):
    a = attr.ib()
    b = attr.ib(repr=False)
    c = attr.ib(repr=lambda value: '***')
print(C(1, 2, 3))
>> C(a=1, c=***)
```

* The moment you want more control over your class initialization than attr.
    * That's when you need a classmethod factory and the builder pattern.
    * If it's just one adjustment after `init` use the `attrs_post_init` hook.
    * Order of Hook Execution is:
        * Defaults -> Converters -> Validators -> Post Hook.

```python
@attr.s
class C(object):
    x = attr.ib()
    y = attr.ib()
    z = attr.ib(init=False)
    def __attrs_post_init__(self):
        self.z = self.x + self.y
        
i = C(x=1, y=2)

print(i)
>> C(x=1, y=2, z=3)
```

* Every attrs- decorated class has an `attrs_atttrs` attribute.
    * This tuple of `attr.Attribute` carries the metadata for each attribute.
    * You can build your own decorators by using the `attrs_attrs` attribute.

```python
import attr
def print_attrs(cls):
    print(cls.__attrs_attrs__)
    return cls

@print_attrs
@attr.s
class C(object):
    a = attr.ib()
    
>> (Attribute(name='a', default=NOTHING, validator=None, repr=True,
              eq=True, order=True, hash=None, init=True, 
              metadata=mappingproxy({}), type=None, converter=None, 
              kw_only=False),)
```

* Remember that `@attr.s` must execute first because it creates the `attrs_attrs`.
    * The decorator execution order starts from the closest one to the class.
    
```python
# This decorator order...
@a
@b
def f():
    pass

# ...is the syntactic sugar for:
def original_f():
    pass
f = a( b( original_f ) )
```

## Type Annotations

* You can associate a type to an attribute with `attr.ib` or with PEP526.
    * If you don't want to use attr.ib you can assign default values.
    * When using default values an `annotations` attribute is created.

```python
# Using attr.ib
@attr.s
class C:
    x = attr.ib(type=int) # Using attr.ib type argument
    y: int = attr.ib() # Using type annotations
        
print(attr.fields(C).x.type)
>> <class 'int'>
print(attr.fields(C).y.type)
>> <class 'int'>


# Using default values
import typing
# Since Auto_Attribs is True every attrib needs a TYPE
@attr.s(auto_attribs=True)
class AutoC:
    # Class variables ARE NOT class attributes
    cls_var: typing.ClassVar[int] = 5
    # Associate an Int List attribute with default value of []
    l: typing.List[int] = attr.Factory(list)
    # Associate an Int Variable with default value of 1
    x: int = 1
    # Associate an Str variable with default value of "Hi"
    foo: str = attr.ib(default="Hi")
    # Associate an Any vairable with default value of None
    bar: typing.Any = None
        
print(AutoC())
>> AutoC(l=[], x=1, foo="Hi", bar=None)
print(AutoC.cls_var)
>> 5
```

* Default values are useful even without type annotations.

```python
@attr.s
class C(object):
    a = attr.ib(default=42)
    b = attr.ib(default=attr.Factory(list))
    c = attr.ib(factory=list)  # Syntactic sugar for above
    d = attr.ib()
    @d.default
    def _any_name_except_a_name_of_an_attribute(self):
        return {}
    
print(C())
>> C(a=42, b=[], c=[], d={})
```

* Note that `default = []` will not do what you may think.
    * This happens even with function and method signatures.
    * That is the reason attrs comes with *default factory* options.
    
```python
@attr.s
class C(object):
    x = attr.ib(default=[])
    
i = C()
j = C()
i.x.append(42)

print(j.x)
>> [42]
```

* Note that `attr.ib` is still needed for advanced features.
    * Advanced features like decorator based default values.
    * Type annotations are specially usefull with `mypy` and `pytype`.
    * Then we can use type annotations to statically check code.

## Note on Hashing

* Objects need to be hashable to be put into a set or used as keys in a dictionary.
    * The *hash* is an integer that represents the content of an object.
    * The integer is implemented inside the `hash` special method.
    * Never set the `attr.s(hash=x)` parameter yourself, let attr do it.
    * To force attr to populate the `hash` method you can:
        * If `frozen=True` then objects will be hashable by value.
        * If `eq=True` then objects will be hashable by their object identity.
    * Collections of objects with big hash codes can be cashed:
        * Assuming attrs has filled the `hash` function, set `cahce_hash=True`.
        

## How it Works

* Attrs is not the first library to simplify class definition.
* However, it has a declarative approach and no runtime overhead.
* When you apply the `attr.s` decorator it searches for `attr.ibs`.
* This class represents the data passed into the arguments `attr.ib`.
* It also includes a counter to preserve the attributes order.
* It also ensures subclassing by searching and collecting base classes.
* This search through  the class hierarchy does not call `super()`.
* Once it has the attributes it writes the requested dunder methods.
* If slotted it adds a new class, if not it adds thems directly.
* By default, the only leftover in the class is the `attrs_attrs` attribute.
* Immutability comes from attaching a special `setattr` method.
* In the end once you start instantiating attrs is long gone.

## Dict and Slotted

* Normally python creates regular classes named `dict class`.
    * Their attributes are stored in the `dict` of every instance.
    * Memory wasteful, specially if you have a lot of inst but few data.
* A slotted class is the memory efficient version of a dict class.
    * Attributes are stored in the special `__slots__` attribute.
    * They use less memory on CPython but have their own special cases.
    * For example they don't allow for other attributes to be set.
    * To maintain their benefits try to inherit from other slotted classes.
    
```python
import attr
@attr.s(slots=True)
class Coordinates(object):
    x = attr.ib()
    y = attr.ib()
c = Coordinates(x=1, y=2)

c.z = 3
>> AttributeError: 'Coordinates' object has no attribute 'z'
```

* Slotted classes must also implement `getstate` and `setstate`.
    * This is so they are serializable with pickle protocol 0 and 1.
* Note that when using PyPy there is no memory advantage of slotted classes.

## API

### Core

1. attr.s: (Apply to the class)
    1. these=None (dict or str to attr.ib)
        1. A dictionary that maps names to attr.ib.
        2. Used to avoid definition of attributes within the class body.
        3. Because you can't directly change the class or don't want to.
        4. If used it will NOT search the class body for attr.ib
    2. repr_ns=None (str)
        1. Explicit way of setting the base repr for nested classes.
    3. repr=True (bool)
        1. Set the attrs human readable representation of the class.
    4. cmp=None (bool or None)
        1. If True is equivalent to `eq=True, order=True`.
        2. Deprecated in favor of eq and order.
    5. hash=None (bool or None)
        1. If None the hash is generated if `eq` and `frozen` are True.
        2. If True attrs forces hash generation. Use at your own risk.
    6. init=True (bool)
        1. Create an `init` method that initializes `attr.ib` attributes.
        2. Leading underscores are stripped from the arguments.
    7. slots=False (bool)
        1. Create a slotted class that is more memory efficient.
    8. frozen=False (bool)
        1. Make instances immutable after initialization.
        2. Creates a custom `setattr` method that raises the error.
        3. You cannot modify self in `post_init` or `init`.
    9. weakref_slot=True (bool)
        1. Make instances *weak referenceable*. Only works with slotted classes.
    10. str=False (bool)
        1. Create a `__str__` method from `repr`. Not necessary, used for Exceptions.
    11. auto_attribs=False (bool)
        1. If True collect annotated attributes from the class body.
        2. Then you must annotate every attribute.
        3. Attributes that are not `attr.ib` or `typing.ClassVar` are ignored. 
    12. kw_only=False (bool)
        1. Make every attribute keyword-only.
    13. cache_hash=False (bool)
        1. For big object hash codes it computes them only once and stores them.
        2. Hashing must be enabled and must not change the hash post creation.
    14. auto_exc=False (bool)
        1. If the class subclasses BaseException.
        2. This will disable some features to behave like an *exceptions class*.
        3. All attributes are available as a tuple, value of str is ignored.
        4. Instances compare and hash by instance IDS.
        5. Values for `eq, order, hash` are ignored.
    15. eq=None (bool or None)
        1. If True or None adds `eq` and `ne` methods that check for equality.
        2. They compare instances of the same class as if they were tuples.
    16. order=None (bool or None)
        1. If None use the `eq` to order instances.
        2. If True use `lt`, `le`, `gt`, `ge`. to order instances.
2. attr.ib: (Apply to the attribute)
     1. default=NOTHING (Any value)
         1. Value used if no value is passed while instantiating.
         2. If Factory its callable will be used to construct the value.
         3. Can be set using decorator notation!.
     2. factory=None (callable)
         1. Syntactic sugar for `default=attr.Factory(Callable)`.
     3. validator=None (callable or a list of callables)
         1. Callable called at init, they receive the initialized instance.
         2. If a list is passed all validators must pass.
     4. repr=True (bool or callable to custom function)
         1. Used to include an attribute in the generated repr.
         2. If False omit the attribute from the repr.
         3. To format the repr str use a callable that returns a string.
     5. eq=None (bool)
         1. If True include `eq` and `ne` for this attribute.
     6. order=None (bool)
         1. If True include `lt`, `le`, `gt`, `ge` for this attribute.
     7. cmp=None (bool)
         1. Deprecated. It is equivalent to `eq=True` and `order=True`.
     8. hash=None (bool or None)
         1. Used to include or ignore this attribute in the generated hash.
     9. init=True (bool)
         1. Used to include or ignore the attribute in the init method.
     10. converter=None (callable)
         1. A callable called at init to convert to the desired format.
         2. It is given the passed in value and returns the desired value.
     11. metadata=None
         1. Arbitrary mapping used by third party components.
     12. type=None
         1. Short hand to specify the type of the attribute.
         2. It is recommended to use type annotation.
     13. kw_only=False
         1. Make this attribute keyword only.
     

3. class attr.Attribute (name, default, validator, repr, cmp, hash, init, metadata=None, type=None, converter=None, kw_only=False, eq=None, order=None)
    1. This is the read-only representation of an attribute.
    2. Pulls the name and all the arguments of an attr.ib.
    3. Except factories since they are syntactic sugar for `default=Factory`.
    4. Used for introspection purposes by other attrs methods and errors.
    
4. attr.make_class(name, attrs, bases=(<class 'object'>, ), **attributes_arguments)
    1. Showthand way of creating a new attrs class.
    
```python
C1 = attr.make_class("C1", ["x", "y"])
print( C1(1, 2) )
>> C1(x=1, y=2)

C2 = attr.make_class("C2", {"x": attr.ib(default=42), "y": attr.ib(default=attr.Factory(list))})
print( C2() )
>> C2(x=42, y=[])
```

5. class attr.Factory(factory, takes_self=False):
    1. Stores a factory callable.
    2. If used as a default it generates the new value at instantiation.
    3. It can use the partially initialized instance with `takes_self=True`.
    
```python
@attr.s
class C(object):
    x = attr.ib(default=attr.Factory(list))
    y = attr.ib(default=attr.Factory( lambda self: set(self.x), takes_self=True) )
    
print( C() )
>> C(x=[], y=set())
    
print( C([1, 2, 3]) )
>> C(x=[1, 2, 3], y={1, 2, 3})
```

6. exception attr.exceptions.FrozenInstanceError
    1. Tried to modify a frozen instance.

7. exception attr.exceptions.AttrsAttributeNotFoundError
    1. An attrs function could not find the attribute that needed.

8. exception attr.exceptions.AttrsAttributeNotFoundError
    1. A non attrs class has been passed into an attrs function.

9. exception attr.exceptions.DefaultAlreadySetError
    1. Tried to set a default with `attr.ib` but also with a decorator.

10. exception attr.exceptions.UnannotatedAttributeError
    1. Class with `auto_attribs=True` has an attribute without type annotation. 

11. exception attr.exceptions.NotCallableError(msg, value)
    1. An `attr.ib` requiring a callable has been set with a not callable.

### Helpers

1. attr.fields(cls)
    1. Return a tuple of attributes of a class.
    2. You can access fields by their assigned names.

```python
@attr.s
    class C(object):
        x = attr.ib()
        y = attr.ib()
        
print( attr.fields(C).y is attr.fields(C)[1] )
>> True
```

2. attr.fields_dict(cls)
    1. Return an ordered dictionary of attributes of a class.
    2. The keys are the attribute names.
    
```python
@attr.s
    class C(object):
        x = attr.ib()
        y = attr.ib()
        
print( attr.fields_dict(C)['y'] is attr.fields(C).y )
>> True
```

3. attr.has(cls)
    1. Return whether a class has `attrs` attributes.
    2. Meaning that it used the attr.s decorator.
 
```python
@attr.s
    class C(object):
        pass
        
print( attr.has(C) )
>> True

print( attr.has(object) )
>> False
```

4. attr.asdict(inst, recurse=True, filter=None, 
    dict_factory=<class 'dict'>, retain_collection_types=False)
    1. Return the attributes of an instance as a dictionary.
    2. Optionally recurse into other hierarchy attrs classes.
    
```python
@attr.s
class C(object):
    x = attr.ib()
    y = attr.ib()

i1 = C(2, 3)
i2 = C(1, i1)
        
print( attr.asdict(i) )
>> {'x': 1, 'y': {'x': 2, 'y': 3}}
```    
    
5. attr.astuple(inst, recurse=True, filter=None, 
    tuple_factory=<class 'tuple'>, retain_collection_types=False)
    1. Return the attributes of an instance as a tuple.
    2. Optionally recurse into other hierarchy attrs classes.
    
```python
@attr.s
class C(object):
    x = attr.ib()
    y = attr.ib()

i1 = C(2, 3)
i2 = C(1, i1)
        
print( attr.astuple(i) )
>> (1, (2, 3))
```

6. attr.filters.include(*what)
    1. Whitelist and include one or a list of attributes.
    2. Can be used with asdict and astuple.
    
7. attr.filters.exclude(*what)
    1. Blacklist and exclude one or a list of attributes.
    2. Can be used with asdict and astuple.
    
8. attr.evolve(inst, **changes)
    1. Create a new instance based on a previews instance.
    2. It also applies some keyword changes in the attributes.
    3. It is the only way to *modify* frozen classes.
    
```python
@attr.s
class C(object):
    x = attr.ib()
    y = attr.ib()
    
i1 = C(1, 2)
i2 = attr.evolve(i1, y=3)
print(i2)
>> C(x=1, y=3)
```

9. attr.validate(inst)
    1. Validate all attributes of the instance that have a validator.
    2. This function is run by default one time at instantiation.
    3. But this allows to recheck after the attributes have changed.

```python
@attr.s
class C(object):
    x = attr.ib(validator=attr.validators.instance_of(int))
    
i = C(1)
i.x = "1"
attr.validate(i)
>> TypeError: "'x' must be <class 'int'> (got '1' that is a <class 'str'>)."
```

* If you fear the performance costs of executing validate:
    * You can easily disable and enable validators globally.
    * Use `attr.set_run_validators(bool)` to set it.
    * And `attr.get_run_validators()` to know if validators are activated.

### Validators

1. attr.validators.instance_of(type)
    1. TypeError if attr.ib is of other type.
    2. Internally it uses `isinstance` so you can pass a tuple of types.
    
```python
# TypeError if x is not an int
@attr.s
class C(object):
    x = attr.ib(validator=attr.validators.instance_of(int))
    
print( C("Hi") )
>> TypeError: "'x' must be <type 'int'> (got '42' that is a <type 'str'>)."
```
2. attr.validators.in_(options)
    1. ValueError if attr.ib is a value other than the provided options.
    2. Options can be a state, list, tuple or enum.
    
```python
>>> import enum
>>> class State(enum.Enum):
...     ON = "on"
...     OFF = "off"
>>> @attr.s
... class C(object):
...     state = attr.ib(validator=attr.validators.in_(State))
...     val = attr.ib(validator=attr.validators.in_([1, 2, 3]))
>>> C(State.ON, 1)
C(state=<State.ON: 'on'>, val=1)
>>> C("on", 1)
Traceback (most recent call last):
   ...
ValueError: 'state' must be in <enum 'State'> (got 'on')
>>> C(State.ON, 4)
Traceback (most recent call last):
   ...
ValueError: 'val' must be in [1, 2, 3] (got 4)
```

3. attr.validators.provides(interface)
    1. TypeError if attr.ib does not provide the interface.
    2. Internally it uses `interface.providedBy(Value)`.
    
4. attr.validators.and_(*validators)
    1. Composes multiple validators into one.
    2. Runs all wrapped validators.
    3. Equivalent to passing a list to the validator argument.
    
5. attr.validators.optional(validator)
    1. Makes an attr.ib optional.
    2. That means that None is an acceptable validator result.
    
6. attr.validators.is_callable()
    1. NonCallableError if attr.ib value is not a callable.
    
```python
@attr.s
class C(object):
    x = attr.ib(validator=attr.validators.is_callable())

# Isinstance is a method and therefore a callable.
print(C(isinstance)) 
>> C(x=<built-in function isinstance>)

# A str is not a callable
print(C("Not a callable"))
>> attr.exceptions.NotCallableError: "'x' must be callable (got 'not a callable' that is a <class 'str'>)."
```
    
7. attr.validators.matches_re(regex, flags=0, func=None)
    1. ValueError if attr.ib does not match a str regex.
    2. It can have additional flags and re-functions.
    3. Re function: re.fullmatch, re.search, re.match, re.fullmatch.
    
```python
@attr.s
class User(object):
    email = attr.ib(validator=attr.validators.matches_re("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)"))

print( User(email="user@example.com") )
>> User(email='user@example.com')

print( User(email="user@example.com@test.com") )
>> ValueError: "'email' must match regex '(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\\\.[a-zA-Z0-9-.]+"
```

8. attr.validators.deep_iterable(member_validator, iterable_validator=None)
    1. TypeError if attr.ib does not perform a deep iterable validation.
    2. For each member of an iterable:
        1. A validation is applied to the iterable members.
        2. An optional validation is applied to the iterable itself.
    
```python
# The iterable must be a list.
# The members must be ints.
@attr.s
class C(object):
    x = attr.ib(validator=attr.validators.deep_iterable(
    member_validator=attr.validators.instance_of(int),
    iterable_validator=attr.validators.instance_of(list)
    ))

print( C(x=[1, 2, 3]) )
>> C(x=[1, 2, 3])
    
print( C(x=set([1, 2, 3]) ) )
>> TypeError: "'x' must be <class 'list'> (got {1, 2, 3} that is a <class 'set'>)."

print( C(x=[1, 2, "3"]) ) 
>> TypeError: ("'x' must be <class 'int'> (got '3' that is a <class 'str'>)."
```

9. attr.validators.deep_mapping(key_validator, value_validator, mapping_validator=None)
    1. Type Error if attr.ib does not perform a deep dictionary validation.
    2. For each member of a dictionary:
        1. A validator is applied to the dictionary keys.
        2. A validator is applied to the dictionary values.
        3. Optional validator is applied to the top level mapping attribute.

```python
# The keys must be strings
# The values must be ints
# The mapping attribute must be a dictionary
@attr.s
class C(object):
    x = attr.ib(validator=attr.validators.deep_mapping(
        key_validator=attr.validators.instance_of(str),
        value_validator=attr.validators.instance_of(int),
        mapping_validator=attr.validators.instance_of(dict)
    ))
    
print( C(x={"a": 1, "b": 2}) )
>> C(x={'a': 1, 'b': 2})

print( C(x={"a": 1.0, "b": 2}) )
>> TypeError: ("'x' must be <class 'int'> (got 1.0 that is a <class 'float'>)."
               
print( C(x={"a": 1, 7: 2}) )
>> TypeError: ("'x' must be <class 'str'> (got 7 that is a <class 'int'>)."
```

### Converters

1. attr.converters.optional(converter)
    1. Makes the `attr.ib` optional.
    2. An optional attribute is one that can be set to None.
    3. If it is not None then it uses the `converter`.
    
```python
# Convert any x value to int.
# But x can also be None
@attr.s
class C(object):
    x = attr.ib(converter=attr.converters.optional(int))

print( C(42) )
>> C(x=42)
print( C(None) )
>> C(x=None)
```
    
2. attr.converters.default_if_none(default=NOTHING, factory=None)
    1. If the `attr.ib` is set to None.
    2. Then convert the attribute to `default` or `factory`.
    
```python
# If x is None convert to default or factory.
@attr.s 
class C(object):
    x = attr.ib(converter=attr.converters.default_if_none(""))

print( C(None) )
>> C(x="")
```